In [43]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [44]:
import re
from functools import partial
from collections import Counter
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import defaultdict
from collections import Counter
from wordcloud import WordCloud
nltk.download('stopwords')
plt.style.use('ggplot')
stop=set(stopwords.words('english'))

def removeUnicode(text):
	#Removes unicode strings like "\u002c" and "x96"
	text = re.sub(r'(\\u[0-9A-Fa-f]+)',r'', text)       
	text = re.sub(r'[^\x00-\x7f]',r'',text)
	return text

def replaceURL(text):
	#Replaces url address with "url" 
	text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
	text = re.sub(r'#([^\s]+)', r'\1', text)
	return text

def replaceAtUser(text):
	#Replaces "@user" with "atUser"
	text = re.sub('@[^\s]+',' ',text)
	return text

def removeHashtag(text):
	#Removes hastag in front of a word
	text = re.sub(r'#([^\s]+)', r'\1', text)
	return text

def removeNumbers(text):
	#Removes integers
	text = ''.join([i for i in text if not i.isdigit()])         
	return text

def replaceMulExcl(text):
	#Replaces repetitions of exlamation marks
	text = re.sub(r"(\!)\1+", '!', text)
	return text

def replaceMulQues(text):
	#Replaces repetitions of question marks
	text = re.sub(r"(\?)\1+", '?', text)
	return text

def replaceMulStop(text):
	#Replaces repetitions of stop marks
	text = re.sub(r"(\.)\1+", '.', text)
	return text

def countMulExcl(text):
	#count repetitions of exlamation marks
	return len(re.findall(r"(\!)\1+", text))

def countMulQues(text):
	#Count repetitions of question marks
	return len(re.findall(r"(\?)\1+", text))

def countMulStop(text):
	#Count repetitions of stop marks
	return len(re.findall(r"(\.)\1+", text))

def countElongated(text):
	#count of how many words are elongated
	regex = re.compile(r"(.)\1{2}")
	return len([word for word in text.split() if regex.search(word)])

def countAllCaps(text):
	#count of how many words are all caps
	return len(re.findall("[A-Z0-9]{3,}", text))

#Creates a dictionary with slangs and their equivalents and replaces them
with open('./temp_files/slang.txt') as file:
	slang_map = dict(map(str.strip, line.partition('\t')[::2])
	for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True)
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)])

#punctuation list for replacing

puncts = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '$', '&', '/', '[', ']', '%', '=', '*', '+', '\\', '•',  '~', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def removePuncts(x):
	x = str(x)
	for punct in puncts:
		if punct in x:
			x = x.replace(punct, f' ')
	return x



def countSlang(text):
	# counts how many slang words and a list of found slangs
	slangCounter = 0
	slangsFound = []
	tokens = nltk.word_tokenize(text)
	for word in tokens:
		if word in slang_words:
			slangsFound.append(word)
			slangCounter += 1
	return slangCounter, slangsFound

#Replaces contractions from a string to their equivalents
contraction_patterns = [ (r'I\'m', 'I am'),(r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'), (r'ain\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), (r'(\w+)n\'t', '\g<1> not'),
						 (r'(\w+)\'ve', '\g<1> have'), (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'), (r'dont', 'do not'), (r'wont', 'will not') ]
def replaceContraction(text):
	patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
	for (pattern, repl) in patterns:
		(text, count) = re.subn(pattern, repl, text)
	return text

def replaceElongated(word):
	#Replaces an elongated word with its basic form

	repeat_regexp = re.compile(r'(\w*)(\w)\2(\w*)')
	repl = r'\1\2\3'
	if wordnet.synsets(word):
		return word
	repl_word = repeat_regexp.sub(repl, word)
	if repl_word != word:      
		return replaceElongated(repl_word)
	else:       
		return repl_word

def removeEmoticons(text):
	#Removes emoticons from text 
	text = re.sub(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', '', text)
	return text

def countEmoticons(text):
	#Input: a text, Output: how many emoticons
	return len(re.findall(':\)|;\)|:-\)|\(-:|:-D|=D|:P|xD|X-p|\^\^|:-*|\^\.\^|\^\-\^|\^\_\^|\,-\)|\)-:|:\'\(|:\(|:-\(|:\S|T\.T|\.\_\.|:<|:-\S|:-<|\*\-\*|:O|=O|=\-O|O\.o|XO|O\_O|:-\@|=/|:/|X\-\(|>\.<|>=\(|D:', text))


### Spell Correction begin ###
def words(text): return re.findall(r'\w+', text.lower())

WORDS = Counter(words(open('./temp_files/spell_correction.txt').read()))

def P(word, N=sum(WORDS.values())): 
	#P robability of `word`.
	return WORDS[word] / N

def spellCorrection(word): 
	#Most probable spelling correction for word.
	return max(candidates(word), key=P)

def candidates(word): 
	#Generate possible spelling corrections for word.
	return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
	#The subset of `words` that appear in the dictionary of WORDS.
	return set(w for w in words if w in WORDS)

def edits1(word):
	#All edits that are one edit away from `word`.
	letters    = 'abcdefghijklmnopqrstuvwxyz'
	splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
	deletes    = [L + R[1:]               for L, R in splits if R]
	transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
	replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
	inserts    = [L + c + R               for L, R in splits for c in letters]
	return set(deletes + transposes + replaces + inserts)

def edits2(word): 
	#All edits that are two edits away from `word`.
	return (e2 for e1 in edits1(word) for e2 in edits1(e1))

### Spell Correction End ###

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kiranbal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
import pandas as pd
import numpy as np
import string
from time import time
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize 


def k_prep(inputText = 'null'):
	'''
	c = input("\n1.Enter or 2.default or 3.df ? :")

	if c == "2":
		text = "AFRICA,#AFRICANBAZE: asap goaaaaal @Breaking !!!! news:Nigeria oooooooh :-D aren't flag???? wont set ablaze..... 12000 in America. http://t.co/2nndBGwyEi,1"
		#print(text)
	
	elif c == "3" :
		text = inputText

	else:
		text = input("\nEnter the tweet: ")
	'''

	text = inputText
	#print("\nReplacing url from tweet\n")
	text = replaceURL(text)
	#print(text)

	#print("\nReplacing atuser from tweet\n")
	text = replaceAtUser(text)
	#print(text)

	#print("\nremoving hashtag in tweet\n")
	text = removeHashtag(text)
	#print(text)

	#print("\nreplace at user in tweet\n")
	text = replaceAtUser(text)
	#print(text)

	'''
	#print("\nremoving stopwords\n")
	#nltk.download('stopwords')
	from nltk.corpus import stopwords
	stop = set(stopwords.words('english'))

	d=[]
	d.append([x for x in text.split() if x not in stop])
	d = d[0]
	text = ' '.join(d)
	#print(text)
	'''

	#print("\nremove numbers from tweet\n")
	text = removeNumbers(text)
	#print(text)

	#print("\nremove emoticons from tweet\n")
	text = removeEmoticons(text)
	#print(text)

	#couting multple punctuations
	#print("\ncounting multiple punctuations\n")
	MultiExclMarks = 0
	MultiQuesMarks = 0
	MultiStopMarks = 0

	MultiExclMarks += countMulExcl(text)
	MultiQuesMarks += countMulQues(text)
	MultiStopMarks += countMulStop(text)

	#print(MultiExclMarks,MultiQuesMarks,MultiStopMarks)

	#print("\nremove multiexclamations from tweet\n")
	text = replaceMulExcl(text)
	#print(text)

	#print("\nremove multiquestionmarks from tweet\n")
	text = replaceMulQues(text)
	#print(text)

	#print("\nremove multistopmarks from tweet\n")
	text = replaceMulStop(text)
	#print(text)


	#print("\nshortening elongated words\n")
	totalElongated = 0
	totalElongated += countElongated(text)
	#print(totalElongated)

	regex1 = re.compile(r"(.)\1{2}")
	l=[]
	for word in text.split():
		if(regex1.search(word)):
			new_word = replaceElongated(word)
			##print(new_word)
			l.append(new_word)
		else:
			l.append(word)
	text = ' '.join(l)
	#print(text)

	#print("\nRemoving punctuations except ?!\n")
	text = removePuncts(text)
	#print(text)
	
	#print("\nexpanding slangs in tweet\n")
	text = replaceSlang(text)
	#print(text)

	#print("\nreplace contractions in tweet\n")
	text = replaceContraction(text)
	#print(text)
	
	#print(\nTokenizing the text\n")
	text = word_tokenize(text)
	
	#print("\nLemmatizing the text\n")
	lemma = WordNetLemmatizer()
	
	list1 = []
	for txt in text:
		list1.append(lemma.lemmatize(txt))
		
	return list1

In [46]:
import pandas as pd
import gensim

nltk.download('punkt')
nltk.download('wordnet')
df = pd.read_csv("../dataset/train.csv")
df['text'] = df['text'].apply(k_prep)

def joinData(list1):
  new_data = " ".join(list1)
  return new_data

df['text'] = df['text'].apply(joinData)
df.to_csv('df_train_preprocessed.csv',index=False)
print(df)

[nltk_data] Downloading package punkt to /home/kiranbal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kiranbal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


         id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this earthquake Ma...       1  
1                 Forest fire near La Ronge Sask Canada       1  
2     All resident asked to shelter in place are bei...       1  
3     people receive wildfire evacuation order in Ca...       1  
4     Just got sent this photo from Ruby Alaska a sm...       1  
...                                                 ...     ...  
7608  Two giant crane holding a bridge collapse into...       1  
7609  The out of control wild fire in California eve...       1  
7610                 

In [47]:
input_df = pd.read_csv('./df_train_preprocessed.csv',usecols=['id','text','target'])
#fixing conflicts
target_error = [328,443,513,2619,3640,3900,4342,5781,6552,6554,6570,6701,6702,6729,6861,7226]
input_df.at[input_df['id'].isin(target_error),'target'] = 0
input_df.head()

,id,text,target
0,1,Our Deeds are the Reason of this earthquake Ma...,1
1,4,Forest fire near La Ronge Sask Canada,1
2,5,All resident asked to shelter in place are bei...,1
3,6,people receive wildfire evacuation order in Ca...,1
4,7,Just got sent this photo from Ruby Alaska a sm...,1


In [48]:
from sklearn import preprocessing
from keras.layers import Input,Dense,Embedding,LSTM,Dropout,Activation
from keras.layers import Bidirectional,GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [49]:
input_df.text.values

array(['Our Deeds are the Reason of this earthquake May ALLAH Forgive u all',
       'Forest fire near La Ronge Sask Canada',
       'All resident asked to shelter in place are being notified by officer No other evacuation or shelter in place order are expected',
       ..., 'M UTC ? km S of Volcano Hawaii',
       'Police investigating after an e bike collided with a car in Little Portugal E bike rider suffered serious non life threatening injury',
       'The Latest More Homes Razed by Northern California Wildfire ABC News'],
      dtype=object)

In [50]:
embedded_size = 100
max_features = 10000
maxlen = 100

In [51]:
test = pd.read_csv('../dataset/test.csv')
test_X = test.text.values


In [52]:
# Split the data to train and validation
from sklearn.model_selection import train_test_split
train_df ,val_df = train_test_split(input_df,test_size = 0.1 , random_state = 43)
train_X = train_df.text.values
val_X = val_df.text.values

In [53]:
# tokenizing the text data using Tokenizer function
tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(train_X))

with open("tokenizer_backup",'wb') as file:
    pickle.dump(tokenizer,file)

train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [17]:
# padding the values
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [18]:
type(test_X)

numpy.ndarray

In [19]:
train_y = train_df.target.values
val_y = val_df.target.values

In [ ]:

#Building layers of embedding
model = tf.keras.Sequential([
     tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length), 
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(6, activation='relu'),
     tf.keras.layers.Dense(1, activation='sigmoid')
])

In [20]:
from keras.models import Model

inp = Input(shape = (maxlen,))
x = Embedding(max_features,embedded_size)(inp)
x = Bidirectional(LSTM(64, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(16,activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(1,activation = 'sigmoid')(x)
model = Model(inputs = inp,outputs = x)
model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

print(model.summary())

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          84480     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropo

In [21]:
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint('./temp_files/best_model2_16bz_preprocessed.h5', monitor='val_accuracy', save_best_only=True)

In [39]:
# training the model
model.fit(train_X, train_y,callbacks=[checkpoint], batch_size=512, epochs=10, validation_data=(val_X, val_y))

Train on 6851 samples, validate on 762 samples
Epoch 1/10
6851/6851 [==============================] - 5s 774us/step - loss: 0.3390 - accuracy: 0.8809 - val_loss: 0.3890 - val_accuracy: 0.8346
Epoch 2/10
6851/6851 [==============================] - 5s 768us/step - loss: 0.2952 - accuracy: 0.8879 - val_loss: 0.4105 - val_accuracy: 0.8373
Epoch 3/10
6851/6851 [==============================] - 5s 762us/step - loss: 0.2697 - accuracy: 0.8994 - val_loss: 0.4103 - val_accuracy: 0.8360
Epoch 4/10
6851/6851 [==============================] - 5s 770us/step - loss: 0.2490 - accuracy: 0.9102 - val_loss: 0.4124 - val_accuracy: 0.8320
Epoch 5/10
6851/6851 [==============================] - 5s 770us/step - loss: 0.2304 - accuracy: 0.9209 - val_loss: 0.4244 - val_accuracy: 0.8333
Epoch 6/10
6851/6851 [==============================] - 5s 772us/step - loss: 0.2142 - accuracy: 0.9275 - val_loss: 0.4395 - val_accuracy: 0.8346
Epoch 7/10
6851/6851 [==============================] - 5s 766us/step - loss:

In [41]:
#preds = model.predict([test_X],batch_size = 1024,verbose = 1)
#preds
import pickle
with open("lstm_model_backup",'wb') as file:
    pickle.dump(model,file)

In [24]:
model.load_weights('./temp_files/best_model2_16bz_preprocessed.h5')
#test_pred = model.predict(test_input)
probabilities = model.predict(test_X)
probabilities

array([[0.6122646 ],
       [0.57825834],
       [0.8766701 ],
       ...,
       [0.82612413],
       [0.76766604],
       [0.55688345]], dtype=float32)

In [25]:
predictions = (probabilities > 0.5).astype(int)
predictions = np.ndarray.flatten(predictions)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [29]:
original_test_df = pd.read_csv("../dataset/test.csv")
print(original_test_df)

         id keyword location  \
0         0     NaN      NaN   
1         2     NaN      NaN   
2         3     NaN      NaN   
3         9     NaN      NaN   
4        11     NaN      NaN   
...     ...     ...      ...   
3258  10861     NaN      NaN   
3259  10865     NaN      NaN   
3260  10868     NaN      NaN   
3261  10874     NaN      NaN   
3262  10875     NaN      NaN   

                                                   text  
0                    Just happened a terrible car crash  
1     Heard about #earthquake is different cities, s...  
2     there is a forest fire at spot pond, geese are...  
3              Apocalypse lighting. #Spokane #wildfires  
4         Typhoon Soudelor kills 28 in China and Taiwan  
...                                                 ...  
3258  EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...  
3259  Storm in RI worse than last hurricane. My city...  
3260  Green Line derailment in Chicago http://t.co/U...  
3261  MEG issues Hazardous Weather 

In [37]:
#filter test set
original_test_df['text'] = original_test_df['text'].apply(k_prep)
original_test_df['text'] = original_test_df['text'].apply(joinData)
print(original_test_df['text'])


0                      Just happened a terrible car crash
1       Heard about earthquake is different city stay ...
2       there is a forest fire at spot pond goose are ...
3                    Apocalypse lighting Spokane wildfire
4               Typhoon Soudelor kill in China and Taiwan
                              ...                        
3258    EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259    Storm in RI worse than last hurricane My city ...
3260                     Green Line derailment in Chicago
3261              MEG issue Hazardous Weather Outlook HWO
3262    CityofCalgary ha activated it Municipal Emerge...
Name: text, Length: 3263, dtype: object


In [38]:
df_final = pd.DataFrame({'text' : original_test_df['text'],'prediction' : predictions,'probabilities' : np.ndarray.flatten(probabilities)})
#df.to_csv("test_df.csv", index=False)
df_final[:60]

,text,prediction,probabilities
0,Just happened a terrible car crash,1,0.612265
1,Heard about earthquake is different city stay ...,1,0.578258
2,there is a forest fire at spot pond goose are ...,1,0.876670
3,Apocalypse lighting Spokane wildfire,0,0.357839
4,Typhoon Soudelor kill in China and Taiwan,1,0.962055
5,We re shaking It s an earthquake,1,0.555409
6,They d probably still show more life than Arse...,0,0.160086
7,Hey ! How are you ?,0,0.180907
8,What a nice hat ?,0,0.281300
9,Fuck off !,0,0.329697


In [ ]:
result = predictions.tolist()
result

In [18]:
submit = pd.read_csv("../dataset/sample_submission.csv")
submit.head()

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0


In [19]:
submit['target'] = result
print(submit)

         id  target
0         0       0
1         2       1
2         3       1
3         9       0
4        11       1
...     ...     ...
3258  10861       1
3259  10865       1
3260  10868       1
3261  10874       1
3262  10875       1

[3263 rows x 2 columns]


In [20]:
submit.to_csv("./temp_files/lstm_submit_callbacks_corrected_16bz.csv",index=False)